In [1]:
from tkinter import *
from PIL import ImageTk
from PIL import Image
import pandas as pd

import cx_Oracle 
con = cx_Oracle.connect("KIHYUK/3927@192.168.0.216:1521/orcl") # 연결 설정


root=Tk()
root.geometry("600x800+100+100")
flag=False
Label(root,text=u"Restrant of Amigo",font=("Helvetica",20)).place(x=170,y=20)
sel_menu1={1:(u"선택안함",0),2:(u"수프",4000),3:(u"바케트",3500),4:(u"케이준샐러드",5500)}
sel_menu2={1:(u"선택안함",0),2:(u"비프스테이크",25000),3:(u"함박스테이크",30000),4:(u"돈까스",15000)}
sel_menu3={1:(u"선택안함",0),2:(u"에스프레소",4000),3:(u"벨기에와플",3500),4:(u"아이스크림",5000)}
sel_menu={u"에피타이저":sel_menu1,u"메인메뉴":sel_menu2,u"디저트":sel_menu3}
final_selected={1:(u"선택안함",0),2:(u"선택안함",0),3:(u"선택안함",0)}
menu_name={1:u"에피타이저",2:u"메인메뉴",3:u"디저트"}
cost=0
class menu:
    def __init__(self,number):
        self.number=number
        self.this_menu_name=menu_name.get(number)
        self.var=IntVar()
        self.var.set(1)
        
    def selected(self):
        temp=sel_menu.get(self.this_menu_name).get(self.var.get())[0]
        Label(root,text=u"선택된 메뉴:").place(x=360,y=70+170*(self.number-1))
        Label(root,text=u"선택된 메뉴:"+temp).place(x=360,y=70+170*(self.number-1))
        global final_selected
        final_selected[self.number]=sel_menu.get(self.this_menu_name).get(self.var.get())
    
    def pop_menu(self):
        global menu_1
        menu_1=Toplevel(root)
        menu_1.title(self.this_menu_name)
        y=170*self.number+25
        #여기서 문제 발생
        menu_1.geometry("250x100+110+%d" % (y))
        for i in sel_menu.get(self.this_menu_name):
            Radiobutton(menu_1,text=sel_menu.get(self.this_menu_name).get(i)[0],variable=self.var,value=i,indicatoron=0,command=self.selected).pack(fill=X)

            
def make_menu(number):
    global this_menu
    this_menu=menu(number)         
    this_menu.pop_menu() 

# def reciept():
    
    
# def sales()


def order():
    global v
    global flag
    ordering=Toplevel(root)
    global cost
    for i in final_selected:
        cost+=final_selected[i][1]
    result = cost
    for i in menu_name:
        Label(ordering,text=menu_name.get(i)+':').grid(row=i,column=1)
        Label(ordering,text=final_selected.get(i)[0]).grid(row=i,column=3)
        Label(ordering,text=final_selected.get(i)[1]).grid(row=i,column=5)
    
    Label(ordering,text=u"총 계산금액=" + str(cost) + u"원 입니다.").grid(row=4,column=2)
#     rows = [("seq.nextval",cost,"sysdate")]
    cursor=con.cursor()
    cursor.execute("INSERT INTO ORDER_MENU(OD_ID,OD_PRICE,OD_DATE) VALUES(seq1.nextval,:ct,sysdate)",ct=cost)
    con.commit()
    df_ora = pd.read_sql("select * from order_menu",con=con)
    df_ora.to_csv('매출')
    con.close()
    return result
    
mymenu=menu(root)
Button(root,text=u"에피타이저",command=lambda: make_menu(1)).place(x=10,y=70)
Button(root,text=u"메인메뉴",command=lambda: make_menu(2)).place(x=10,y=210)  
Button(root,text=u"디저트",command=lambda: make_menu(3)).place(x=10,y=350)
Button(root,text=u"영수증출력",command=order).place(x=400,y=650)
Button(root,text=u"주문하기",command=order).place(x=500,y=650)

images=[(1,"1.jpg"),(2,"2.jpg"),(3,"3.jpg")]

for i,image in images:
    img=Image.open(image)
    this_image=ImageTk.PhotoImage(img)
    mylabel=Label(image=this_image)
    mylabel.image=this_image
    mylabel.place(x=100,y=70+(i-1)*130)
root.config(width=600,height=800,background="seashell3")
root.title(u"Restaurant of Amigo")
root.mainloop()    

# print(cost)

DatabaseError: ORA-12170: TNS: 접속 시간 초과가 발생함